In [ ]:
!pip install -q \
    langgraph==0.2.* \
    langchain==0.3.* \
    langchain-openai \
    langchain-tavily \
    numpy \
    scikit-learn \
    scipy \
    pydantic<2  # совместимость с FlagEmbedding

# Установим FlagEmbedding (официальный пакет для bge-m3)
!pip install -q "FlagEmbedding>=1.3.0"

# 📦 ИМПОРТЫ
import os, warnings, numpy as np
from datetime import datetime
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from sklearn.preprocessing import normalize
from FlagEmbedding import BGEM3FlagModel

warnings.filterwarnings("ignore", category=UserWarning, module="langchain")

# 🔑 key.env
if not os.path.exists('key.env'):
    raise FileNotFoundError("❌ Загрузите key.env через 📁 → Upload")
with open('key.env', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line and '=' in line and not line.startswith('#'):
            k, v = line.split('=', 1)
            os.environ[k.strip()] = v.strip()
if not (os.getenv("OPENAI_API_KEY") and os.getenv("TAVILY_API_KEY")):
    raise ValueError("❌ Ключи не найдены")

# 🧠 ИНИЦИАЛИЗАЦИЯ — BGE-M3 через FlagEmbedding (гибридный поиск)
print("🧠 Загружаю BAAI/bge-m3 через FlagEmbedding (гибридный поиск)…")
emb_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')  # fp16=False для CPU

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
search_tool = TavilySearch(max_results=4)

ALLOWED_PROGRAMS = ["Python", "R", "Julia", "MATLAB", "SPSS", "SAS", "Stata", "Wolfram Mathematica", "JASP", "PSPP"]

# 📌 ИНДЕКСАЦИЯ: dense + sparse
print("🔍 Индексирую программы…")
embeddings = emb_model.encode(
    ALLOWED_PROGRAMS,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False
)
print("🔍 Индексирую программы…")
embeddings = emb_model.encode(
    ALLOWED_PROGRAMS,
    return_dense=True,
    return_sparse=True,
    return_colbert_vecs=False
)

dense_vecs = normalize(np.array(embeddings["dense_vecs"]), norm='l2')
sparse_vecs = embeddings.get("sparse_vecs")

if sparse_vecs is None:
    print("⚠️ Sparse embeddings недоступны — использую dense-only поиск.")
    use_hybrid = False
else:
    print("✅ Sparse embeddings получены — включаю гибридный поиск.")
    use_hybrid = True

def hybrid_search(query: str, top_k=3, dense_weight=0.7):
    q_emb = emb_model.encode(
        [query],
        return_dense=True,
        return_sparse=True,
        return_colbert_vecs=False
    )
    q_dense = normalize(np.array(q_emb["dense_vecs"]), norm='l2')[0]
    dense_scores = dense_vecs @ q_dense

    if use_hybrid and sparse_vecs is not None:
        q_sparse = q_emb.get("sparse_vecs", [None])[0]
        if q_sparse is not None:
            sparse_scores = sparse_vecs.dot(q_sparse.T).toarray().flatten()

            def _norm(x):
                if x.max() == x.min():
                    return np.zeros_like(x)
                return (x - x.min()) / (x.max() - x.min() + 1e-8)
            dense_scores = _norm(dense_scores)
            sparse_scores = _norm(sparse_scores)
            hybrid = dense_weight * dense_scores + (1 - dense_weight) * sparse_scores
            scores = hybrid
        else:
            scores = dense_scores
    else:
        scores = dense_scores

    top_idx = np.argsort(scores)[::-1][:top_k]
    return [ALLOWED_PROGRAMS[i] for i in top_idx]

# 📁 Отчёты (HTML + MD)
def save_reports(program: str, report_text: str, results: list):
    os.makedirs("reports", exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = f"report_{program.lower().replace(' ', '_')}_{ts}"

    html_items = []
    for r in results[:3]:
        title = r.get("title", "N/A")
        url = r.get("url", "#")
        content = r.get("content", "")[:200].replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        html_items.append(f"<div><strong><a href='{url}' target='_blank'>{title}</a></strong><br>{content}…</div>")
    html = f"""<!DOCTYPE html><html lang="ru"><head><meta charset="utf-8"><title>{program}</title>
<style>body{{font-family:Arial,sans-serif;line-height:1.6;max-width:800px;margin:40px auto}}
.report{{background:#f9f9f9;padding:20px;border-radius:5px;white-space:pre-wrap}}</style></head>
<body><h1>📊 Отчёт: <code>{program}</code></h1><p><i>{datetime.now().strftime('%d.%m.%Y %H:%M')}</i></p>
<div class="report">{report_text}</div><h2>🔍 Источники</h2>{''.join(html_items)}</body></html>"""
    html_path = f"reports/{base}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html)

    md_items = []
    for r in results[:3]:
        title = r.get("title", "N/A")
        url = r.get("url", "#")
        content = r.get("content", "")[:120].strip().replace("\n", " ")
        md_items.append(f"- **[{title}]({url})** — {content}…")
    md = f"# Отчёт по `{program}`\n\n> {datetime.now().strftime('%d.%m.%Y %H:%M')}\n\n{report_text}\n\n---\n## 🔍 Источники\n" + "\n".join(md_items)
    md_path = f"reports/{base}.md"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(md)

    print(f"✅ Отчёты:\n   📄 {html_path}\n   📝 {md_path}")
    return html_path, md_path

# 🧠 Граф
class AgentState(TypedDict):
    query: str; candidates: List[str]; confirmed: str; html_path: str; md_path: str

def find_program(state):
    candidates = hybrid_search(state["query"])
    print(f"🔎 Гибридный поиск: {', '.join(candidates)}")
    return {"candidates": candidates}

def confirm_program(state):
    print(f"\n👉 Рекомендуемые: {', '.join(state['candidates'])}")
    confirm = input("Введите точное название: ").strip()
    for p in ALLOWED_PROGRAMS:
        if confirm.lower() == p.lower() or confirm.lower() in p.lower():
            print(f"✅ Подтверждено: {p}")
            return {"confirmed": p}
    fallback = state["candidates"][0]
    print(f"⚠️ Использую: {fallback}")
    return {"confirmed": fallback}

def generate_report(state):
    raw = search_tool.invoke(f"Что такое {state['confirmed']}? применение в DS")
    results = raw.get("results", raw) if isinstance(raw, dict) else raw
    if not isinstance(results, list):
        results = [{"title": "❌", "url": "#", "content": "Ошибка"}]
    report = llm.invoke([("human", f"Кратко на русском: что такое {state['confirmed']}, где применяется в DS, лицензия. Без markdown.")]).content.strip()
    html, md = save_reports(state["confirmed"], report, results)
    return {"html_path": html, "md_path": md}

builder = StateGraph(AgentState)
builder.add_node("find", find_program)
builder.add_node("confirm", confirm_program)
builder.add_node("report", generate_report)
builder.set_entry_point("find")
builder.add_edge("find", "confirm")
builder.add_edge("confirm", "report")
builder.add_edge("report", END)
graph = builder.compile()

# ▶️ Запуск
query = input("💬 О чём отчёт? > ")
final = graph.invoke({"query": query, "candidates": [], "confirmed": "", "html_path": "", "md_path": ""})

/bin/bash: line 1: 2: No such file or directory
🧠 Загружаю BAAI/bge-m3 через FlagEmbedding (гибридный поиск)…


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


🔍 Индексирую программы…
🔍 Индексирую программы…
⚠️ Sparse embeddings недоступны — использую dense-only поиск.
💬 О чём отчёт? > ррр
🔎 Гибридный поиск: R, SPSS, PSPP

👉 Рекомендуемые: R, SPSS, PSPP
Введите точное название: R
✅ Подтверждено: R
✅ Отчёты:
   📄 reports/report_r_20251217_162604.html
   📝 reports/report_r_20251217_162604.md
